In [ ]:
# Uninstall existing bitsandbytes and install the latest version
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes

Found existing installation: bitsandbytes 0.45.4
Uninstalling bitsandbytes-0.45.4:
  Successfully uninstalled bitsandbytes-0.45.4
  Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl (76.0 MB)


In [ ]:
# Install other required packages
!pip install -U transformers accelerate sentencepiece

In [ ]:
!pip install protobuf==3.20.3 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:

# Print GPU info for debugging
!nvidia-smi

# Verify bitsandbytes installation
!python -c "import bitsandbytes; print(f'bitsandbytes version: {bitsandbytes.__version__}')"

/bin/bash: line 1: nvidia-smi: command not found
Could not load bitsandbytes native library: /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbyte

In [ ]:
# === IMPORTS ===
import os
import pandas as pd
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm.notebook import tqdm
import gc
import logging
from codecarbon import EmissionsTracker

In [ ]:
# === CONFIG ===
logging.getLogger("codecarbon").setLevel(logging.ERROR)

input_path = '/content/drive/MyDrive/Dataset_LLM/Prompt_Dataset.csv'
output_dir = '/content/drive/MyDrive/Dataset_LLM/batch_outputs/'
merged_output = '/content/drive/MyDrive/Dataset_LLM/Prompt_Dataset_with_mistral.csv'
log_file = '/content/drive/MyDrive/Dataset_LLM/inference_log.txt'
batch_size = 20
prompt_column = 'Prompt'

os.makedirs(output_dir, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# === LOGGING ===
def log(msg):
    with open(log_file, 'a') as f:
        f.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {msg}\n")
    print(msg)

In [ ]:
# === LOAD MISTRAL MODEL ===
def load_mistral_model():
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv("HUGGINGFACE_TOKEN"))
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def clear_cuda():
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    try:
        log("Trying 4-bit quantization...")
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )
        clear_cuda()
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto",
            low_cpu_mem_usage=True
        )
    except Exception as e4:
        log(f"4-bit failed: {e4}")
        try:
            log("Trying 8-bit quantization...")
            quant_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)
            clear_cuda()
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quant_config,
                device_map="auto",
                low_cpu_mem_usage=True
            )
        except Exception as e8:
            log(f"8-bit failed: {e8}")
            log("Falling back to fp16...")
            clear_cuda()
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                low_cpu_mem_usage=True
            )

    return model, tokenizer

In [ ]:
# === GENERATE CLEAN RESPONSE ===
def generate_mistral_response(prompt, model, tokenizer):
    mistral_prompt = f"<s>[INST] {prompt} [/INST]"
    inputs = tokenizer(mistral_prompt, return_tensors="pt").to(model.device)

    tracker = EmissionsTracker(measure_power_secs=1, log_level="error")
    tracker.start()
    start_time = time.time()

    output = model.generate(
        inputs.input_ids,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1048
    )

    emissions = tracker.stop()
    duration = time.time() - start_time

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=False).replace("<s>", "").strip()
    response = decoded_output.split('[/INST]', 1)[1].strip() if '[/INST]' in decoded_output else decoded_output.strip()

    if response.startswith(prompt.strip()):
        response = response[len(prompt.strip()):].strip()

    token_length = len(output[0]) - len(inputs.input_ids[0])
    return response, token_length, duration, emissions

In [ ]:
# === MAIN GENERATION PROCESS ===
def process_large_dataset():
    log("Loading dataset...")
    df = pd.read_csv(input_path)

    for col in ['Mistral_Response', 'Response_Token_length', 'Response_Total_duration', 'Energy_Consumption_kWh']:
        if col not in df.columns:
            df[col] = None

    model, tokenizer = load_mistral_model()
    total_rows = len(df)
    processed_batches = set(
        int(f.split('_')[-1].replace('.csv', '')) for f in os.listdir(output_dir) if f.startswith('batch_'))

    for start in range(0, total_rows, batch_size):
        batch_id = (start // batch_size) + 1
        batch_path = os.path.join(output_dir, f'batch_{batch_id}.csv')

        if batch_id in processed_batches:
            log(f"Skipping already processed batch {batch_id}")
            continue

        end = min(start + batch_size, total_rows)
        batch = df.iloc[start:end].copy()
        log(f"Processing batch {batch_id}: rows {start} to {end - 1}")

        for i in tqdm(batch.index):
            prompt = batch.loc[i, prompt_column]
            if pd.isna(prompt):
                continue
            try:
                response, token_len, duration, emissions = generate_mistral_response(prompt, model, tokenizer)
                batch.at[i, 'Mistral_Response'] = response
                batch.at[i, 'Response_Token_length'] = token_len
                batch.at[i, 'Response_Total_duration'] = duration
                batch.at[i, 'Energy_Consumption_kWh'] = emissions
            except Exception as e:
                log(f"Error at row {i}: {e}")

            if torch.cuda.is_available() and i % 3 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        batch.to_csv(batch_path, index=False)
        log(f"Saved batch {batch_id} to {batch_path}")

    del model, tokenizer
    gc.collect()

    log("Merging all batches...")
    full_df = pd.read_csv(input_path)
    for col in ['Mistral_Response', 'Response_Token_length', 'Response_Total_duration', 'Energy_Consumption_kWh']:
        if col not in full_df.columns:
            full_df[col] = None

    for f in sorted(os.listdir(output_dir)):
        if f.startswith('batch_') and f.endswith('.csv'):
            batch_df = pd.read_csv(os.path.join(output_dir, f))
            full_df.update(batch_df)

    full_df.to_csv(merged_output, index=False)
    log(f"Final merged output saved to {merged_output}")

In [ ]:
# === RUN ===
process_large_dataset()

Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Saved batch 181 to /content/drive/MyDrive/Dataset_LLM/batch_outputs/batch_181.csv
Processing batch 182: rows 3620 to 3639


  0%|          | 0/20 [00:00<?, ?it/s]

Saved batch 182 to /content/drive/MyDrive/Dataset_LLM/batch_outputs/batch_182.csv
Processing batch 183: rows 3640 to 3659


  0%|          | 0/20 [00:00<?, ?it/s]